# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Подготовка

In [1]:
import os
import json
import string
import numpy as np
import pandas as pd
from math import log
from judicial_splitter import splitter
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec, KeyedVectors, Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim import matutils

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocessing(text, mode):
    morph = MorphAnalyzer()
    stops = stopwords.words('russian')
    punct = string.punctuation + '«»“”—…'
    
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    all_lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    
    if mode == 'd2v':
        cleaned_lemmas = [lemma for lemma in all_lemmas if lemma not in punct]
    else:
        cleaned_lemmas = [lemma for lemma in all_lemmas if lemma not in stops and lemma not in punct]
 
    return cleaned_lemmas

In [3]:
corpus = os.listdir('article')[:10]
corpus = ['article/' + file for file in corpus]

# Индексация

## Обратный индекс

In [4]:
def corpus_and_stats(corpus):
    prep_corpus = []
    doc_lengths = {}

    for filename in corpus:
        with open(filename, 'r', encoding='utf-8') as file:
            file = file.read()

        text = preprocessing(file, 'ii')
        doc_lengths[filename] = len(text)

        prep_corpus.append(' '.join(text))
    
    avgdl = sum(doc_lengths.values()) / len(doc_lengths)
    
    return prep_corpus, doc_lengths, avgdl

In [5]:
def calc_idfs(term_doc_matrix):    
    idfs = {}

    docs = term_doc_matrix[0]
    words = term_doc_matrix[1]
    numbers = term_doc_matrix[2]

    N = len(docs)

    for i, word in enumerate(words):
        n = 0

        for num in numbers:
            val = num[i]

            if val != 0:
                n += 1

        idf = log((N - n + 0.5) / (n + 0.5))
        idfs[word] = idf
    
    return idfs

In [6]:
def inverted_index(corpus):
    prep_corpus, doc_lengths, avgdl = corpus_and_stats(corpus)
    
    vectorizer = CountVectorizer()
    docs = vectorizer.fit_transform(prep_corpus)

    term_doc_matrix = pd.DataFrame(docs.toarray(), columns=vectorizer.get_feature_names(), index=corpus)
    term_doc_matrix = [term_doc_matrix.index.tolist(), term_doc_matrix.columns.tolist(), term_doc_matrix.values.tolist()]
    
    idfs = calc_idfs(term_doc_matrix)
    
    return term_doc_matrix, doc_lengths, avgdl, idfs

## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [ ]:
model = Word2Vec.load('araneum_none_fasttextcbow_300_5_2018.model')

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [1]:
def get_w2v_vectors(paragraph):
    counter = 0
    hypervector = np.zeros(300)
    stops = stopwords.words('russian')
    
    for word in paragraph:
        try:
            vector = np.array(model.wv[word])
            hypervector += vector
            counter += 1
        except:
            continue
            
    hypervector = hypervector / counter
    
    return hypervector


def save_w2v_base(corpus):
    base = []

    for filename in corpus:
        with open(filename, 'r', encoding='utf-8') as file:
            file = file.read()

        paragraphs = splitter(file, 4)

        for par in paragraphs:
            prep_par = preprocessing(par, 'w2v')
            vector = get_w2v_vectors(prep_par)
            base.append([vector.tolist(), filename])
            
    with open('jud_base_w2v.json', 'w', encoding='utf-8') as file:
        json.dump(base, file, allow_nan=False)

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [121]:
class D2VIterator(object):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __iter__(self):
        for i, text in enumerate(self.texts):
              yield LabeledSentence(text, [self.labels[i]])

In [79]:
def train_doc2vec(d2v_iterator, model_name):
    model = Doc2Vec(vector_size=300, min_count=0, alpha=0.025, min_alpha=0.025)
    model.build_vocab(d2v_iterator)

    for epoch in range(100):
        model.train(d2v_iterator, total_examples=model.corpus_count, epochs=model.epochs)
        model.alpha -= 0.002
        model.min_alpha = model.alpha
        model.save(model_name)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [142]:
def get_d2v_vectors(corpus):
    prep_corpus, labels = [], []

    for filename in corpus:
        with open(filename, 'r', encoding='utf-8') as file:
            file = file.read()

        paragraphs = splitter(file, 4)

        for i, par in enumerate(paragraphs):
            prep_par = preprocessing(par, 'd2v')
            prep_corpus.append(' '.join(text))
            labels.append(filename)
        
    d2v_iterator = D2VIterator(prep_corpus, labels)
    train_doc2vec(d2v_iterator, 'doc2vec.model')
    trained_model = Doc2Vec.load('doc2vec.model')
    
    return trained_model, labels


def save_d2v_base(corpus):
    trained_model, labels = get_d2v_vectors(corpus)
    base = []
    
    for label in labels:
        vector = trained_model.docvecs[label]
        base.append([vector.tolist(), label])
        
    with open('jud_base_d2v.json', 'w', encoding='utf-8') as file:
        json.dump(base, file)

# Прогоняем

In [85]:
#inverted index
term_doc_matrix, doc_lengths, avgdl, idfs = inverted_index(corpus)

In [88]:
#word2vec
save_w2v_base(corpus)

In [151]:
#doc2vec
save_d2v_base(corpus)

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

In [124]:
k1 = 2.0
b = 0.75

def score_BM25(idf, freq, D, avgdl) -> float:
    score = idf * (k1 + 1) * freq / (freq + k1 * (1 - b + b * D / avgdl))
    
    return score

Функция измерения близости между векторами нам пригодится:

In [125]:
def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [147]:
def search_w2v(request):
    vec_request = get_w2v_vectors(request)    
    results = {}
    
    with open('jud_base_w2v.json', 'r', encoding='utf-8') as file:
        base = json.load(file)
    
    for elem in base:
        res = similarity(vec_request, elem[0])
        results[elem[1]] = res
    
    return sorted(results.items(), key=lambda kv: kv[1], reverse=False)
      
    
def search_d2v(request):
    vec_request = trained_model.infer_vector(request)    
    results = {}
    
    with open('jud_base_d2v.json', 'r', encoding='utf-8') as file:
        base = json.load(file)
    
    for elem in base:
        res = similarity(vec_request, elem[0])
        results[elem[1]] = res    
    
    return sorted(results.items(), key=lambda kv: kv[1], reverse=False)


def search_inv_index(request):
    results = {doc: 0 for doc in term_doc_matrix[0]}
    
    for word in request:
        if word in term_doc_matrix[1]:
            word_ind = term_doc_matrix[1].index(word)
            
            for i, doc in enumerate(term_doc_matrix[0]):
                freq = term_doc_matrix[2][i][word_ind]
                D = doc_lengths[doc]
                idf = idfs[word]      
                bm25 = score_BM25(idf, freq, D, avgdl)
                
                results[doc] += bm25
    
    return sorted(results.items(), key=lambda kv: kv[1], reverse=True)
                

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [127]:
def search(request, search_method):
    if search_method == 'inverted_index':
        request = preprocessing(request, 'ii')
        search_result = search_inv_index(request)
        
    elif search_method == 'word2vec':
        request = preprocessing(request, 'w2v')
        search_result = search_w2v(request)
        
    elif search_method == 'doc2vec':
        request = ' '.join(preprocessing(request, 'd2v'))
        search_result = search_d2v(request)
    else:
        raise TypeError('unsupported search method')
    
    return search_result
    

In [ ]:
results = search('утка охота соболь лицензия ружье', 'inverted_index')
results[:5]

In [ ]:
results = search('утка охота соболь лицензия ружье', 'word2vec')
results[:5]

In [152]:
results = search('утка охота соболь лицензия ружье', 'doc2vec')
results[:5]